In [1]:
import rawpy
import numpy as np
import os
import glob
import sys
import yaml
from imageio import imwrite
from PIL import Image, ExifTags
from astropy.io import fits

In [ ]:
# Constants
folder = '20241107'
image_x = 4000
image_y = 6000

In [3]:
# Load config
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

dark_seconds = config['dark_seconds']
print('Dark seconds:', dark_seconds)
dark_folders = [f'dark_{ds}' for ds in dark_seconds]
print('Dark folders:', dark_folders)

Dark seconds: [1, 2, 5, 10]
Dark folders: ['dark_1', 'dark_2', 'dark_5', 'dark_10']


In [9]:
def binning_process(image: np.ndarray, binning:int):
    # Binning
    length = image.shape[0] // binning
    width = image.shape[1] // binning
    image = image.reshape((length, binning, width, binning, 3))
    image = np.mean(image, axis=(1, 3))
    return image

In [ ]:
# Refactorize the process to compute the dark mean as a function
def compute_dark_mean(folder, dark_folder, image_x:int, image_y:int):
    dark_frames_names = glob.glob(f'{folder}/{dark_folder}/*.CR3')
    print('Dark frames:', dark_frames_names)
    binning = 2
    length = image_x // binning
    width = image_y // binning
    dark_frames = np.zeros((len(dark_frames_names), length, width, 3), dtype=np.uint16)
    for idx, dark_frame_name in enumerate(dark_frames_names):
        with open(dark_frame_name, 'rb') as f:
            dark_frame = rawpy.imread(f)
            dark_frame = dark_frame.postprocess() # 8 bits
            dark_frame = binning_process(dark_frame, binning)
            dark_frame = dark_frame.astype(np.uint16)
            # Convert from gamma=2.2 to linear
            dark_frame = (dark_frame / 255.0) ** 2.2 * 65535
            dark_frames[idx] = dark_frame
    dark_mean = np.mean(dark_frames, axis=0)
    dark_mean = dark_mean.astype(np.uint16)

    dark_mean = np.moveaxis(dark_mean, -1, 0)
    print(dark_mean.shape)
    # Save the image as a fits file
    hdu = fits.PrimaryHDU(dark_mean)
    hdul = fits.HDUList([hdu])
    hdul.writeto(f'{folder}/{dark_folder}/dark_mean.fits', overwrite=True)


In [11]:
# Compute the dark mean for each folder
for df in dark_folders:
    compute_dark_mean(folder, df, image_x, image_y)

Dark frames: ['20241107/dark_1/IMG_0175.CR3', '20241107/dark_1/IMG_0174.CR3', '20241107/dark_1/IMG_0176.CR3', '20241107/dark_1/IMG_0177.CR3', '20241107/dark_1/IMG_0179.CR3', '20241107/dark_1/IMG_0178.CR3', '20241107/dark_1/IMG_0183.CR3', '20241107/dark_1/IMG_0182.CR3', '20241107/dark_1/IMG_0180.CR3', '20241107/dark_1/IMG_0181.CR3']
(3, 2000, 3000)
Dark frames: ['20241107/dark_2/IMG_0167.CR3', '20241107/dark_2/IMG_0173.CR3', '20241107/dark_2/IMG_0172.CR3', '20241107/dark_2/IMG_0166.CR3', '20241107/dark_2/IMG_0170.CR3', '20241107/dark_2/IMG_0164.CR3', '20241107/dark_2/IMG_0165.CR3', '20241107/dark_2/IMG_0171.CR3', '20241107/dark_2/IMG_0168.CR3', '20241107/dark_2/IMG_0169.CR3']
(3, 2000, 3000)
Dark frames: ['20241107/dark_5/IMG_0158.CR3', '20241107/dark_5/IMG_0159.CR3', '20241107/dark_5/IMG_0161.CR3', '20241107/dark_5/IMG_0160.CR3', '20241107/dark_5/IMG_0162.CR3', '20241107/dark_5/IMG_0163.CR3', '20241107/dark_5/IMG_0154.CR3', '20241107/dark_5/IMG_0155.CR3', '20241107/dark_5/IMG_0157.CR3'

In [13]:
# Step 2: Compute the dark subtracted image

In [14]:
image_list = [f'{folder}/image/IMG_{i:04d}.CR3' for i in range(246, 267)]
print(image_list)

['20241107/image/IMG_0246.CR3', '20241107/image/IMG_0247.CR3', '20241107/image/IMG_0248.CR3', '20241107/image/IMG_0249.CR3', '20241107/image/IMG_0250.CR3', '20241107/image/IMG_0251.CR3', '20241107/image/IMG_0252.CR3', '20241107/image/IMG_0253.CR3', '20241107/image/IMG_0254.CR3', '20241107/image/IMG_0255.CR3', '20241107/image/IMG_0256.CR3', '20241107/image/IMG_0257.CR3', '20241107/image/IMG_0258.CR3', '20241107/image/IMG_0259.CR3', '20241107/image/IMG_0260.CR3', '20241107/image/IMG_0261.CR3', '20241107/image/IMG_0262.CR3', '20241107/image/IMG_0263.CR3', '20241107/image/IMG_0264.CR3', '20241107/image/IMG_0265.CR3', '20241107/image/IMG_0266.CR3']


In [ ]:
def subtract_image(image_list:list, exptime:str):
    for image_name in image_list:
        try:
            with open(image_name, 'rb') as f:
                image = rawpy.imread(f)
                image = image.postprocess()
                image = binning_process(image, 2)
        except:
            print(f'Error loading {image_name}')
            continue
        image = image.astype(np.int16)
        # Convert from gamma=2.2 to linear
        image = (image / 255.0) ** 2.2 * 65535
        # Move the last dimension to the first one
        image = np.moveaxis(image, -1, 0)

        # Laod the dark mean
        dark_mean = fits.getdata(f'{folder}/dark_{exptime}/dark_mean.fits')

        # Subtract the dark mean from the image
        image_reduced = image - dark_mean
        image_reduced = np.clip(image_reduced, 0, 2**16 - 1)
        image_reduced = image_reduced.astype(np.uint16)
        # Save the image as a fits file
        hdu = fits.PrimaryHDU(image_reduced)
        hdul = fits.HDUList([hdu])
        hdul.writeto(f'{image_name[:-4]}_reduced.fits', overwrite=True)

In [16]:
subtract_image(image_list, '10')

FileNotFoundError: [Errno 2] No such file or directory: '20241107/image/IMG_0257.CR3'